In [42]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize

pd.set_option('max_columns', 50)

In [5]:
# conexión con  mongo
client = MongoClient("mongodb://localhost:27017/")

# conexión con la base de datos "db_companies"
db = client.db_companies

#coleción companies
collection_companies = db.companies

# query pidiendo TODO
query = collection_companies.find()

# convertimos a pandas.DataFrame
df = pd.DataFrame(query)

In [35]:
# vemos el resultado
df.head()

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,deadpooled_month,deadpooled_url,deadpooled_year,description,email_address,external_links,founded_day,founded_month,founded_year,funding_rounds,homepage_url,image,investments,ipo,milestones,name,number_of_employees,offices,overview,partners,permalink,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8a,"{'price_amount': 30000000, 'price_currency_cod...",[],,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,8,2007-05-25 06:51:27,http://www.crunchbase.com/company/wetpaint,NaN,NaN,NaN,1.0,Technology Platform Company,info@wetpaint.com,[{'external_url': 'http://www.geekwire.com/201...,17.0,10.0,2005,"[{'id': 888, 'round_code': 'a', 'source_url': ...",http://wetpaint-inc.com,"{'available_sizes': [[[150, 75], 'assets/image...",[],NaN,"[{'id': 5869, 'description': 'Wetpaint named i...",Wetpaint,47,"[{'description': '', 'address1': '710 - 2nd Av...",<p>Wetpaint is a technology platform company t...,[],abc2,206.859.6300,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...",[],"[{'is_past': False, 'title': 'Co-Founder and V...","[{'available_sizes': [[[150, 86], 'assets/imag...","wiki, seattle, elowitz, media-industry, media-...",$39.8M,BachelrWetpaint,Sun Dec 08 07:15:44 UTC 2013,[]
1,52cdef7c4bab8bd675297d91,"{'price_amount': None, 'price_currency_code': ...",[],,http://blog.geni.com/index.rdf,http://blog.geni.com,web,2,Thu May 31 19:52:34 UTC 2007,http://www.crunchbase.com/company/geni,NaN,NaN,None,NaN,Geneology social network site,,[],1.0,6.0,2006,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",http://www.geni.com,"{'available_sizes': [[[150, 60], 'assets/image...",[],None,"[{'id': 15460, 'description': 'Announced hire ...",Geni,18,"[{'description': 'Headquarters', 'address1': '...",<p>Geni is an online community of casual and e...,[],geni,,"[{'name': 'Geni', 'permalink': 'geni'}]",[],"[{'is_past': False, 'title': 'CEO', 'person': ...",[],"geni, geneology, social, family, genealogy",$16.5M,geni,Wed Oct 10 14:01:29 UTC 2012,"[{'embed_code': '<object width=""425"" height=""3..."
2,52cdef7c4bab8bd675297d93,None,"[{'price_amount': 20000000, 'price_currency_co...",None,,,web,0,Thu May 31 21:46:57 UTC 2007,http://www.crunchbase.com/company/fox-interact...,NaN,NaN,,NaN,,,[],1.0,6.0,1979,[],http://www.newscorp.com,"{'available_sizes': [[[150, 71], 'assets/image...","[{'funding_round': {'round_code': 'b', 'source...",None,[],Fox Interactive Media,0,"[{'description': '', 'address1': '407 N Maple ...",<p>Fox Interactive Media (FIM) oversees <a hre...,[],fox-interactive-media,,[],[],"[{'is_past': False, 'title': '', 'person': {'f...","[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,twitterapi,Mon Aug 19 17:13:27 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl..."
3,52cdef7c4bab8bd675297d9d,"{'price_amount': None, 'price_currency_code': ...",[],None,http://blog.joost.com/atom.xml,http://blog.joost.com,games_video,11,Sat Jun 09 07:05:30 UTC 2007,http://www.crunchbase.com/company/joost,NaN,NaN,None,NaN,None,newyork@joost.com,[],1.0,10.0,2006,"[{'id': 19, 'round_code': 'a', 'source_url': '...",http://joost.com,"{'available_sizes': [[[150, 64], 'assets/image...",[],None,"[{'id': 4860, 'description': 'Joost Video Netw...",Joost,0,"[{'description': '', 'address1': '100 5th Ave ...","<p>As their third major undertaking, <a href=""...",[],joost,,"[{'name': 'Joost', 'permalink': 'joost'}]",[],"[{'is_past': False, 'title': 'CEO', 'person': ...",[],"iptv, babelgum, television, video, thevenicepr...",$45M,None,Fri Mar 15 23:31:11 UTC 2013,"[{'embed_code': '<object width=""425"" height=""3..."
4,52cdef7c4bab8bd675297da9,None,[],,http://blog.mahalo.com/feed/,http://blog.mahalo.com/,web,9,Thu Jun 14 03:42:20 UTC 2007,http://www.crunchbase.com/company/mahalo,NaN,NaN,None,NaN,Life long learning con

In [9]:
df.shape

(18801, 42)

In [ ]:
# Filtramos las empresas que cumplen: son de videjuegos, más de 500 empleados, abiertas antes de 2012, 
# y no en USA por problemas políticos


In [24]:
companies = collection_companies.find({"$and":[{"number_of_employees": {"$lt": 50}}, 
                                               {"founded_year": {"$lt": 2009}}, 
                                               {"$or": [{"category_code":"design"}, 
                                                        {"category_code":"games_video"}, 
                                                        {"category_code":"web"} 
                                                       ] 
                                               } ]}, 
                                      {''}
                                     )# no en USA por problemas políticos


In [25]:

lst = list(companies)


In [26]:
len(lst)

2043

In [37]:
df = pd.DataFrame(lst)

In [39]:
df.head()
columns_to_count = ['competitions', 'alias_list', 'competitions', 'competitions' ,'funding_rounds', 'image', 
                   'investments', 'milestones', 'partners', 'products', 'providerships', 'relationships', 
                   'screenshots', 'video_embeds']


In [41]:
def count_elements(x): 
    try: 
        return len(x)
    except: 
        return 0

def clean_lists(dataframe, cols): 
    for column in cols: 
        dataframe[column] = dataframe[column].apply(count_elements)
    return dataframe

df = clean_lists(df, columns_to_count)
df.head()

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,deadpooled_month,deadpooled_url,deadpooled_year,description,email_address,external_links,founded_day,founded_month,founded_year,funding_rounds,homepage_url,image,investments,ipo,milestones,name,number_of_employees,offices,overview,partners,permalink,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8a,"{'price_amount': 30000000, 'price_currency_cod...",[],0,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,0,2007-05-25 06:51:27,http://www.crunchbase.com/company/wetpaint,NaN,NaN,NaN,1.0,Technology Platform Company,info@wetpaint.com,[{'external_url': 'http://www.geekwire.com/201...,17.0,10.0,2005,3,http://wetpaint-inc.com,1,0,NaN,2,Wetpaint,47,"[{'description': '', 'address1': '710 - 2nd Av...",<p>Wetpaint is a technology platform company t...,0,abc2,206.859.6300,2,0,17,1,"wiki, seattle, elowitz, media-industry, media-...",$39.8M,BachelrWetpaint,Sun Dec 08 07:15:44 UTC 2013,0
1,52cdef7c4bab8bd675297d91,"{'price_amount': None, 'price_currency_code': ...",[],0,http://blog.geni.com/index.rdf,http://blog.geni.com,web,0,Thu May 31 19:52:34 UTC 2007,http://www.crunchbase.com/company/geni,NaN,NaN,None,NaN,Geneology social network site,,[],1.0,6.0,2006,3,http://www.geni.com,2,0,None,1,Geni,18,"[{'description': 'Headquarters', 'address1': '...",<p>Geni is an online community of casual and e...,0,geni,,1,0,13,0,"geni, geneology, social, family, genealogy",$16.5M,geni,Wed Oct 10 14:01:29 UTC 2012,3
2,52cdef7c4bab8bd675297d93,None,"[{'price_amount': 20000000, 'price_currency_co...",0,,,web,0,Thu May 31 21:46:57 UTC 2007,http://www.crunchbase.com/company/fox-interact...,NaN,NaN,,NaN,,,[],1.0,6.0,1979,0,http://www.newscorp.com,2,2,None,0,Fox Interactive Media,0,"[{'description': '', 'address1': '407 N Maple ...",<p>Fox Interactive Media (FIM) oversees <a hre...,0,fox-interactive-media,,0,0,66,3,,$0,twitterapi,Mon Aug 19 17:13:27 UTC 2013,3
3,52cdef7c4bab8bd675297d9d,"{'price_amount': None, 'price_currency_code': ...",[],0,http://blog.joost.com/atom.xml,http://blog.joost.com,games_video,0,Sat Jun 09 07:05:30 UTC 2007,http://www.crunchbase.com/company/joost,NaN,NaN,None,NaN,None,newyork@joost.com,[],1.0,10.0,2006,1,http://joost.com,2,0,None,1,Joost,0,"[{'description': '', 'address1': '100 5th Ave ...","<p>As their third major undertaking, <a href=""...",0,joost,,1,0,19,0,"iptv, babelgum, television, video, thevenicepr...",$45M,None,Fri Mar 15 23:31:11 UTC 2013,2
4,52cdef7c4bab8bd675297da9,None,[],0,http://blog.mahalo.com/feed/,http://blog.mahalo.com/,web,0,Thu Jun 14 03:42:20 UTC 2007,http://www.crunchbase.com/company/mahalo,NaN,NaN,None,NaN,Life long learning content,jason@mahalo.com,[],1.0,3.0,2007,2,http://mahalo.com,2,0,None,0,Mahalo,40,"[{'description': '', 'address1': '3525 Eastham...",<p>Mahalo is a human powered search engine fou...,0,mahalo,,1,0,13,0,"search, search-engine, human-powered-search, aaa",$21M,MahaloDotCom,Fri May 17 04:34:19 UTC 2013,1
